# A first tutorial

In this notebook, we describe how to use *crossfolium*.

In [1]:
import pandas as pd
import numpy as np
import folium
import branca


import sys
sys.path.insert(0,'..')
import crossfolium

First, we'll create a set of data with two qualitative columns (*letter* and *greek*), and two quatitative columns (*lat* and *lng*).

In [2]:
N = 10000

In [3]:
data = pd.DataFrame(index=range(N))
data['letter'] = list(map(
    lambda x: 'A' if x < 0.2 else 'B' if x<0.6 else 'C',
    np.random.uniform(size=N)
    ))
data['greek'] =  list(map(
    lambda x: 'Alpha' if x < 0.2 else 'Beta' if x<0.6 else 'Gamma',
    np.random.uniform(size=N)
    ))
data['lat'] = 35. + 20*np.random.uniform(size=N)
data['lng'] = -10.+ 40*np.random.uniform(size=N)

Now, we create a `Figure` object and we'll put things into it.

In [4]:
f = branca.element.Figure(height=1000)

The first thing we put into it is a `Crossfilter` object that will store the data and enable future widgets to have access to it.

In [5]:
c = crossfolium.Crossfilter(data.to_dict(orient='records')).add_to(f)

In the crossfilter, we create a div that will spread over the first quarter of the screen (`2,2,1` means *draw a grid of 2 rows and 2 columns and select the first cell*).

In [6]:
div = c.add_subplot(2,2,1, margin=0)

In this first div, we create a `folium.Map` object. In the map, we create a `HeatmapFilter` object that will draw the data in the form of a heatmap. Note that `HeatmapFilter` contains a reference to the crossfilter `c` so that it will update each time a filter is changed.

In [7]:
div.add_child(
    folium.Map([45,10], zoom_start=4, tiles='cartodbpositron').add_child(
        crossfolium.HeatmapFilter(c, radius=5, blur=5)
        ))
pass

Now we add two `PieFilter` objects for filtering qualitative data.

In [8]:
c.add_subplot(4,4,3, margin=0).add_child(crossfolium.PieFilter(
    c,
    'letter',
    name='letter',
    order=['A','B','C'],
    colors=['#ff0000','#00ff00', '#0000ff'],
    ))

c.add_subplot(4,4,4, margin=0).add_child(crossfolium.PieFilter(
    c,
    'greek',
    name='greek',
    order=['Alpha', 'Beta', 'Gamma'],
    colors=['#00ffff','#ff00ff', '#ffff00'],
    ))
pass

And two `BarFilter` objects for filtering quantitative data.

In [9]:
c.add_subplot(8,2,6, margin=0).add_child(
    crossfolium.BarFilter(
        c,
        'lat', width=400, height=100,
        bar_padding=0.1, domain=[35,55],
        groupby=1,
        elastic_y=True,
        xlabel='',
        ylabel='latitude',
        margins={'top': 10, 'right': 20, 'bottom': 20, 'left': 50},
        xticks=[35,45,55],
        ))

c.add_subplot(8,2,8, margin=0).add_child(
    crossfolium.BarFilter(
        c,
        'lng', width=400, height=100,
        bar_padding=0.1, domain=[-10,30],
        groupby=2,
        elastic_y=True,
        xlabel='',
        ylabel='longitude',
        margins={'top': 10, 'right': 20, 'bottom': 20, 'left': 50},
        xticks=[-10,0,10,20,30],
        ))
pass

At last, we add a `TableFilter` object to have a table representation of the data.

In [10]:
c.add_subplot(2,1,2, margin=0).add_child(
    crossfolium.TableFilter(c, data.columns.values.tolist()
    ))
pass

We now render the figure:

In [11]:
f